<head><b><center>CIENCIA DE DATOS</center>

<center>APRENDIZAJE SUPERVISADO</center>
<center>Regresión y Clasificación</center></b>
    

<center>Profesor: Gabriel Jara </center></head><br>
El presente Jupyter Notebook busca:
<ul>
    <li>Recordar algunos conceptos estadísticos básicos para el aprendizaje automático (Machine Learning).</li>
    <li>Relacionar los problemas de Regresión y Clasificación con técnicas de análisis de datos comunes de las Estadísticas.</li>
    <li>Presentar una aproximación de la Ciencia de Datos a los problemas de Clasificación (Regresión quedará pendiente).</li>    
  <li>Extender nuestro dominio sobre herramientas disponibles en Python para la gestión de datos en general.</li>
</ul>

Nota: Las imágenes que han sido robadas de internet son enlaces a su correspondiente fuente. 



<b>ESTADÍSTICOS BÁSICOS </b>

<b>Varianza y Covarianza</b>

En estadísticas estudió que para una variable aleatoria $x$, puede medir su tendencia central con alguna media, y la dispersión de dicha variable calculando su varianza o desviación estándar.  
$$ \sigma^2 = \frac{\sum_{i=1}^{n}(x_i - \mu)^2} {n} $$

Supongamos que tenemos dos variables aleatorias $x$ e $y$, cada una tendrá su propia tendencia central y dispersión que las caracteriza. Pero puede ocurrir que el comportamiento de ambas variables tenga algún parecido. Podemos medir la dispersión cruzada, es decir como varían en conjunto, ambas variables, usando la covarianza. 



$$ S_{xy} = \frac{\sum_{i=1}^{n}(x_i - \mu_{x})*(y_i - \mu_{y})} {n} $$

A continuación probaremos estas formulas en la práctica. 



In [ ]:
def mi_media_aritmetica(x):
    return sum(x)/len(x)

# Las siguientes funciones se podrían hacer más sintéticas, pero se
# mantendrán simples para mayor legibilidad respecto a la fórmula. 
def mi_varianza(x):
    n = len(x)
    x_media = mi_media_aritmetica(x)
    suma = 0
    for i in x:
        suma += (i-x_media)**2
    varianza = suma/n
    return varianza

def mi_covarianza(x,y):
    if len(x) != len(y):
        print("Error, tamaño de vectores no coinciden.")
        return
    n = len(x)
    x_media = mi_media_aritmetica(x)
    y_media = mi_media_aritmetica(y)
    suma = 0
    for i in range(n):
        suma += (x[i]-x_media)*(y[i]-y_media)
    covarianza = suma/n
    return covarianza

In [ ]:
import numpy as np

# Datos de prueba
x = np.arange(10, 20)
y = np.array([2, 1, 4, 5, 8, 12, 18, 25, 96, 48])
print("x = ", x)
print("y = ", y)
print("La Covarianza calculada con nuestras funciones sería: ")
print(mi_covarianza(x,y))

<b>Correlación</b>

Pero la covarianza es difícil de interpretar, puesto que no corresponde a la escala de ninguna de las dos variables analizadas. Por eso se estandariza la covarianza, para obtener un coeficiente de correlación del que sí podamos saber previamente en qué rango se mueve, facilitando la interpretación y comparación para distintas variables. El más utilizado es el <b>Coeficiente de Correlación de Pearson</b>. 

$$ \rho_{xy} = \frac{S_{xy} } {\sigma_{x}*\sigma_{y}} $$

A continuación definimos una función para calcular la correlación de Pearson. 

In [ ]:
def mi_correlacion_pearson(x,y):
    covarianza = mi_covarianza(x,y)
    x_desv = (mi_varianza(x))**(1/2)
    y_desv = (mi_varianza(y))**(1/2)
    correlacion = covarianza/(x_desv*y_desv)
    return correlacion

In [ ]:
print("x = ", x)
print("y = ", y)
print("La Correlación de Pearson calculada con nuestra formula es: ")
print(mi_correlacion_pearson(x,y))

Normalmente usted preferirá usar módulos como <i>scipy.stats</i> en lugar de definir sus propias funciones de covarianza o correlaciones, pero para efectos de esta demostración nos pareció más apropiado así. 

Existe otros coeficientes de correlación, adecuados según las características de los datos (tipos de datos, distribuciones de probabilidad). Hemos recordado el coeficiente de correlación de Pearson como una forma de ilustrar la naturaleza de la asociación que ocurre entre variables aleatorias.  

<b>REGRESIÓN </b>

<b>Regresión Lineal</b>

Un modelo de Regresión Lineal usa una o varias variables independientes, representadas como $x$, para predecir mediante una función lineal el valor de una variable dependiente, representada como $y$. 

Cuando $x$ es una única variable, lo podemos graficar en un típico gráfico de dos dimensiones, usando los ejes $x$ e $y$, como se ve en la figura. 
<br> <img src=https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1526911465/1_afqiyx.png width="400"/>

El desafío es encontrar los pesos de la ecuación lineal, es decir el valor de las constantes que definen la posición de la recta, que representaremos como $ \beta $. Para ello usamos una muestra de datos de valores $x$ e $y$ conocidos, con los que se ajusta estos valores $ \beta $ de forma de <b>minimizar el error cuadrado</b> del valor predicho respecto a los valores reales de $y$. Este ajuste se realiza utilizando operaciones matriciales que probablemente ya vio en Estadísticas y no necesitará en esta asignatura. La razón para usar error al cuadrado, en lugar de por ejemplo el valor absoluto del error, tiene que ver con la resolución matemática del problema de encontrar los hiperparámetros beta óptimos.  

$$ \hat{y} = \beta_{0} + \beta_{1}*x $$

Una vez que tenemos una recta que ajusta bien a los datos, podemos usarla para estimar el valor $y$ que correspondería a nuevos valores $x$. 

Usaremos el archivo "Ciclo_Combinado.csv" que contiene datos de una planta de energía del tipo Ciclo Combinado, cuya potencia eléctrica generada fue registrada junto a factores que se presume relacionados, como son: temperatura, vacío, presión y humedad ambiental. 

In [ ]:
import numpy as np  
import pandas as pd

# Importar datos desde csv
file = open("Ciclo_Combinado.csv", "r")
obs = file.read()
file.close()
obs = obs.split("\n")
# hay un salto de línea demás en el archivo
obs.remove(obs[-1])
# dividir los datos por coma. 
for j in range(len(obs)):
    obs[j] = obs[j].split(",")
# transformar a numpy array.
obs = np.array(obs)
print('Número de observaciones: ',len(obs))

# Convertir a un DataFrame para mostrar en formato tabular
df_obs = pd.DataFrame(obs)

# Mostrar los primeros 5 registros para un formato más limpio
print(df_obs.head())


Se separa la variable dependiente...

In [ ]:
# Valores Y (variable dependiente)
y = obs[:,-1][1:]
print(y)

... y las variables independientes, o predictoras. Dado que ahora trabajaremos con una matriz en lugar de un vector (tenemos varios $x$ en lugar de una única variable), la convención es representarla con mayúscula: $X$. 

In [ ]:
# Matriz X (variables independientes)
X = obs[:,:-1][1:]
print(X)

Usaremos el módulo Sklearn.linear_model, en particular el objeto LinearRegression que ejecuta la Regresión Lineal para construir un modelo que estima el valor de $y$, usando los valores $X$.  

In [ ]:
from sklearn.linear_model import LinearRegression

# Los valores de X e y tienen que ser numéricos
X, y = np.array(X, dtype='float64'), np.array(y, dtype='float64')

#Se define el modelo
model = LinearRegression().fit(X, y)
r_sq = model.score(X, y)
print('Coeficiente de determinación (R2):', r_sq)
print('Intercepto (Beta0):', model.intercept_)
print('Pendiente (Betas 1 en adelante):', model.coef_)

Sklearn retorna en Coeficiente de Determinación (R2) del modelo. Entre más alto sea este coeficiente, mejor se ajustan los datos al modelo lineal, es decir, los datos estarían bien representados por el modelo. 

$$R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}$$

Aplicado a modelos lineales, el coeficiente R2 normalmente se mueve de 0 a 1, dode 0 es un modelo que no explica los datos observados mejor de lo que ya lo hace la media de los datos, mientras que 1 sería un modelo que los explica perfectamente. Puede darse el caso de un R2 negativo, eso sería un modelo que explica los datos peor aún que asumir que la variable es una constante (con valor igual a su media), lo cual es una situación extrema que no se daría normalmente.   

Además del R2, el resultado del modelo también identifica los valores Beta, incluido el intercepto y pendiente de cada variable $X$ que conforman la línea recta con que se estima los valores $y$. Podríamos usar estos valores para construir la ecuación que genera valores de $\hat{y}$ (predicciones de $y$), pero no es necesario puesto que el módulo ya provee un método para ello. 

In [ ]:
import random
samp = random.sample(range(len(y)),10)
# y con sombrero son los resultados de la predicción
y_hat = model.predict(X)
# imprimieromos sólo una muestra de casos al azar. 
# Crear un diccionario con los resultados reales y predichos para la muestra
data = {
    'Resultado Real': [y[i] for i in samp],
    'Resultado Predicho': [y_hat[i] for i in samp]
}

# Convertir el diccionario a un DataFrame de pandas para mostrar en tabla
df_results = pd.DataFrame(data)

# Mostrar la tabla
print(df_results)

Ahora podemos usar estos resultados como estimación de potencia generada que debiera esperar la planta de ciclo combinado a partir de los datos predictores disponibles. Si tenemos los datos $X$ de una nueva observación compuesta de: temperatura, vacío, presión y humedad, podemos estimar la potencia generada usando el modelo lineal.

Este es un ejemplo típico de un problema de Regresión, y la técnica que hemos aplicado, Regresión Lineal, es la metodología básica que proveen las estadísticas para resolverlo. 

<b>CLASIFICACIÓN </b>

<b>Problema de Clasificación</b>

Pero no todos los problemas son iguales, no siempre es conveniente una regresión lineal. Un caso distinto, por ejemplo, sería si el resultado $y$ no fuera un valor numérico sino categórico. Esto se denomina un Problema de Clasificación.  

Por ejemplo, luego del hundimiento del Titanic los pasajeros pueden haber sobrevivido o no sobrevivido. No hay lugar para estar <i> medio muerto</i> ni <i> medio vivo</i>, tampoco podríamos estar <i> el doble de muerto</i> que otra persona... ¿Es posible predecir este resultado usando los datos de "clase", "género", "edad", "# conyugues y hermanos", "# padres e hijos" y "tarifa pagada"?

In [ ]:
import numpy as np  

file = open("Titanic.csv", "r")
titanic = file.read()
file.close()
titanic = titanic.split("\n")
for j in range(len(titanic)):
    titanic[j] = titanic[j].split(",")
# El nombre tiene ',' así que necesitamos volver a unirlo.
for elemento in titanic[1:]:
    elemento[3] = elemento[3]+elemento[4]
    del elemento[4]
titanic = np.array(titanic)

# Convertir a un DataFrame para mostrar en formato tabular
df_obs = pd.DataFrame(titanic)

# Mostrar los primeros 5 registros para un formato más limpio
print(df_obs.head())


Podemos intentar predecir la supervivencia usando una Regresión Lineal, pero ya debiáramos suponer que este problema no es muy lineal. Sin embargo, Regresión Lineal requiere usar datos numéricos y tenemos algunos que no lo son, así que deberíamos primero transformar todo a una codificación numérica o eliminar atributos que no sean numéricos. En este caso optamos por eliminar lo que no sea número, excepto por el género, que codificaremos como 0-1. 


In [ ]:
# codificando sexo por 0 (masculino) y 1 (feminino)
titanic = np.delete(titanic, [3,8,10,11], axis=1)
for linea in titanic[1:]:
    if linea[3]=='male':
        linea[3] = '0'
    else:
        linea[3] = '1'

Tenemos valores perdidos, lo cual no permite ejecutar los métodos estadísticos que estamos probando. Tendríamos que imputar valores perdidos o eliminarlos. Lo segundo podría ser por dos vías: quitar atributos (columnas) que contienen perdidos o quitar observaciones (filas). En este caso haremos lo segundo, eliminaremos de la muestra pasajeros cuyos datos no están completos. 

In [ ]:
# Eliminando perdidos.
borrar = []
for l in range(len(titanic)):
    if '' in titanic[l]:
        borrar.append(l)
titanic = np.delete(titanic, borrar, axis=0)

Separamos las variables predictoras de la variable dependiente. 

In [ ]:
# Valores y (variable dependiente)
# Estado de supervivencia
y = titanic[:,1][1:]

In [ ]:
# Variables independientes X
# Todos los datos sin considerar Id ni supervivencia
X = titanic[:,2:][1:]

Y aplicamos Regresión Lineal con la ingenua esperanza de que hará un buen pronóstico. 

In [ ]:
# Los valores de X e y tienen que ser numéricos
X, y = np.array(X, dtype='float64'), np.array(y, dtype='float64')

#Se define y entrena el modelo
model = LinearRegression().fit(X, y)
r_sq = model.score(X, y)
print('Coeficiente de determinación (R2):', r_sq)
print('Intercepto (Beta0):', model.intercept_)
print('Pendiente (Betas 1 en adelante):', model.coef_)

El coeficiente de determinación R2 es bajo. Las predicciones serán números reales que no necesariamente se acotan a valores 0 o 1, por lo que no tenemos una interpretación directa. El problema nace de que estamos tratando de ajustar una línea recta a un problema que no es lineal. 

<b>Regresión Logística</b>

Una regresión Logística ajusta un modelo cuyos resultados no son una línea recta, sino que tiene la forma de la función logística, algo así como una S que sube rápidamente, pasado un umbral de 0 a 1. 

$$ \hat{Y} = \frac{1} { 1 + e^{-( \beta_{0} + \beta_{1}*X)}} $$

<br> <img src=https://miro.medium.com/max/1428/1*Vd9ZTC1zWJPtV7iXPMJk1Q.png width="400"/>


In [ ]:
from sklearn.linear_model import LogisticRegression

# Los valores de X e y tienen que ser numéricos
X, y = np.array(X, dtype='float64'), np.array(y, dtype='float64')

#Se define y entrena el modelo
model = LogisticRegression().fit(X, y)
score = model.score(X, y)
print('Exactitud de la clasificación:', score)
print('Intercepto (Beta0):', model.intercept_)
print('Pendiente (Betas 1 en adelante):', model.coef_)

Note que la métrica que viene prevista para evaluar la regresión logística ya NO es el coeficiente de determinación R2, sino que la exactitud de la clasificación. Hablaremos sobre métricas para evaluar los modelos en futura unidad. 

¿Cómo se ve nuestras predicciones? Tomamos una muestra: 

In [ ]:
y_hat = model.predict(X)
samp = random.sample(range(len(y)),20)
# Crear un diccionario con los resultados reales y predichos para la muestra
data = {
    'Resultado Real': [y[i] for i in samp],
    'Resultado Predicho': [y_hat[i] for i in samp]
}

# Convertir el diccionario a un DataFrame de pandas para mostrar en tabla
df_results = pd.DataFrame(data)

# Mostrar la tabla
print(df_results)

Podemos resumir la información de las predicciones, correctas e incorrectas, usando una Matriz de Confusión. Esta consiste en cuatro cuadrantes, identificando las cuatro posibles combinaciones Realidad-Modelo: Muere-Muere, Vive-Muere, Muere-Vive y Vive-Vive.
<br> <img src=https://miro.medium.com/v2/resize:fit:1218/1*jMs1RmSwnYgR9CsBw-z1dw.png width="400"/>

<u>Cuidado</u>: La disposición de matriz de confusión que acá se presenta corresponde a la que observaremos en sklearn a continuación. Sin embargo, el orden del positivo y negativo puede cambiar según la fuente que consulte. Siempre procure verificar que esté interpretando la matriz en el orden correcto, según sea la herramienta con la que la haya obtenido. 


In [ ]:
from sklearn.metrics import confusion_matrix
# Se crea la matriz de confusión. 
matriz_conf = confusion_matrix(y, y_hat)
print('MATRIZ DE CONFUSIÓN:')
print(matriz_conf,'\n')
print('BIEN CLASIFICADOS: ', matriz_conf[0][0]+matriz_conf[1][1])
print('\tPasajeros con sobrevivencia = 0 clasificados como 0: ', matriz_conf[0][0])
print('\tPasajeros con sobrevivencia = 1 clasificados como 1: ', matriz_conf[1][1])
print('MAL CLASIFICADOS: ', matriz_conf[0][1]+matriz_conf[1][0])
print('\tPasajeros con sobrevivencia = 0 clasificados como 1: ', matriz_conf[0][1],'(falso positivo)')
print('\tPasajeros con sobrevivencia = 1 clasificados como 0: ', matriz_conf[1][0],'(falso negativo)')

¿Podríamos obtener una mejor clasificación? Dijimos que Regresión Logística es un modelo predictivo No Lineal, en el sentido de que su predicción no es un valor en una línea recta. Sin embargo, Regresión Logística SÍ es un clasificador lineal.

Al igual que con otros clasificadores lineales, la predicción de Regresión Logística significa quedar a uno u otro lado de una línea que separa las clases. Y si bien la línea recta no es la predicción, implica que los datos solo pueden ser separados por una frontera lineal y recta. Regresión Logística es la técnica estadística que encuentra la mejor línea recta que separa a los datos en un problema de regresión, al igual que Regresión Lineal encuentra la mejor recta que aproxima a los datos en un problema de regresión.   
<br> <img src=https://wirelesspi.com/wp-content/uploads/2022/08/figure-logistic-regression-decision-boundary-2.png width="300"/>

Entonces, con Regresión Logística sólo podríamos llegar a clasificar correctamente datos que sean linealmente separables. Pero en Ciencia de Datos solemos encontrar problemas de clasificación que no son linealmente separables. 

<br> <img src=https://ds100.org/course-notes-su23/logistic_regression_2/logistic_reg_2_files/figure-html/cell-11-output-1.png width="400"/>

<b>CLASIFICACIÓN EN CIENCIA DE DATOS </b>

<b> Árboles de Decisión (Decision Trees)</b>

Son un tipo de modelo de predicción utilizado en diversos ámbitos que van desde la Inteligencia Artificial hasta la Economía. Dado un conjunto de datos se fabrican diagramas de construcciones lógicas, muy similares a los sistemas de predicción basados en reglas, que sirven para representar y categorizar una serie de condiciones que ocurren de forma sucesiva, para la resolución de un problema.
<br> <img src=https://miro.medium.com/v2/resize:fit:541/0*l4TP8pfEmY6aM2vO.png width="400"/>

En cada nodo de decisión se identifica el atributo que permita dividir lo más posible las clases de la población que llega a dicho nodo. En un nodo terminal (hoja) ya no se divide, y las observaciones que llegan a ese nodo se clasifican de acuerdo con la moda de los que allí lleguen. 
<br> <img src=https://cdn-images-1.medium.com/v2/resize:fit:1100/0*ssOP-2Nu5PzJCzN0.png width="600"/>


Usemos datos de prueba que ya están disponibles en sklearn. En este caso aprovecharemos el set de datos llamado <i>iris</i> que contiene 150 observaciones de las características de 3 variedades de flor iris. El objetivo es clasificar de qué variedad es cada flor usando las medidas de pétalos y sétalos. 

Usando un módulo disponible en sklearn, construimos el modelo tipo Árbol e Decisión para clasificación y lo entrenamos con todos los datos disponibles. 

In [ ]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier 

# Descargando el set de datos de prueba
iris = datasets.load_iris()
X = iris.data
y = iris.target
# Definición y entrenamiento del modelo de clasificación
clf = DecisionTreeClassifier(random_state=1234)
model = clf.fit(X, y)

Obtenemos una representación en texto del modelo que podemos imprimir.  

In [ ]:
from sklearn import tree
# Iprimir por pantalla el árbol resultante.
text_representation = tree.export_text(clf)
print(text_representation)

Podemos plotear una mejor representación del modelo. 

In [ ]:
from matplotlib import pyplot as plt
# plotear una visualización del árbol. 
plt.figure(figsize=(25,20))
tree.plot_tree(clf, feature_names=iris.feature_names, 
               class_names=iris.target_names, filled=True)
plt.show()

Observando el nodo raíz del árbol, notamos que si el pétalo NO tiene largo menor o igual a 2.45, ya podemos decir que es de clase setosa. Las 50 observaciones de iris setosa que hay en la muestra cayeron en ese nodo. Las 100 observaciones restantes se clasifican en el sub-árbol que se forma hacia la derecha del nodo raíz, llegando a una profundidad total de 5 niveles. Si analizamos los nodos terminales, veremos que en cada hoja del árbol sólo hay grupos <i>limpios</i>, es decir con observaciones de sólo una categoría. Por lo tanto, el árbol separó completamente los datos según su clase. 

Nuestro problema no era clasificar flores, sino decidir el estado de vida o muerte de los pasajeros del Titanic. Usemos un modelo de tipo Árbol de Decisión para lograrlo. 

Si bien, en teoría, los árboles de decisión pueden separar datos categóricos y numéricos por igual, la computación para separar datos nominales es más compleja y no se encuentra disponible en Python (hasta donde el autor de este apunte (usando Google y ChatGPT) pudo encontrar). Más adelante discutiremos como podemos aprovechar los datos nominales de los que disponemos, pero por ahora seguiremos usando el set de datos que ya habíamos limpiado de datos categóricos y de observaciones con atributos perdidos. 

<b>Nota al Margen: Preprocesamiento como entregable</b>

Previamente cargamos y preprocesamos los datos del Titanic. Esto requirió leer el archivo, eliminar algunos atributos que no usaremos, transformar a representación nominal numérica atributos nominales, entre muchas actividades de limpieza y preparación que podríamos hacer. ¿Debieramos replicar todo ese código en cada ocasión que quiera volver a cargar el set de datos? 

En la clase sobre preparación de los datos vimos como un entregable el proceso de transformación de los mismos datos. Vamos a empaquetar dicho proceso, haciéndolo fácilmente reproducible, como un <b>pipeline</b>. Para esto, al menos por ahora, simplemente usaremos funciones de Python. 

In [ ]:
import pandas as pd
import numpy as np

# Función para cargar el CSV
def cargar_datos(path):
    return pd.read_csv(path)


# Función para eliminar columnas innecesarias
def eliminar_columnas(df):
    return df.drop(columns=['Name', 'Ticket', 'Cabin', 'Embarked'])

# Función para transformar género a valores numéricos
def codificar_sexo(df):
    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
    return df

# Función para eliminar filas con valores faltantes
def eliminar_nulos(df):
    return df.dropna()

# Función principal que aplica todo el pipeline y extrae X e y
def preprocesar_titanic(path):
    df = cargar_datos(path)
    df = eliminar_columnas(df)
    df = codificar_sexo(df)
    df = eliminar_nulos(df)
    
    titanic_array = df.to_numpy()

    # Separamos la columna 1 como y, y desde la columna 2 en adelante como X, excluyendo el encabezado
    y = titanic_array[1:, 1]
    X = titanic_array[1:, 2:]

    return X, y

# Uso del pipeline
X, y = preprocesar_titanic("Titanic.csv")
print("X shape:", X.shape)
print("y shape:", y.shape)


Ahora, cada vez que queramos reproducir el preprocesamiento de los datos, usamos el pipeline en lugar de replicar todo el código. 

Declaramos y entrenamos el modelo. 

In [ ]:
clf = DecisionTreeClassifier(random_state=1234)
model = clf.fit(X, y)
text_representation = tree.export_text(clf)
print(text_representation)

In [ ]:
plt.figure(figsize=(100,40))
tree.plot_tree(clf, feature_names=["clase", "género", "edad", 
                                   "conyugues-hermanos", "padres-hijos" , "tarifa"], 
               class_names=["No sobrevice", "Sí sobrevive"], filled=True)
plt.show()

¿Qué tan bueno será este modelo? Podemos comparar su exactitud con la que habíamos obtenido usando Regresión Logística. 

In [ ]:
# Exactitud
r_sq = model.score(X, y)
print('Exactitud:', r_sq)


Aparentemente está mucho mejor, clasifica mejor las observaciones.

In [ ]:
# y con sombrero
y_hat = model.predict(X)
samp = random.sample(range(len(y)),20)
# Crear un diccionario con los resultados reales y predichos para la muestra
data = {
    'Resultado Real': [y[i] for i in samp],
    'Resultado Predicho': [y_hat[i] for i in samp]
}

# Convertir el diccionario a un DataFrame de pandas para mostrar en tabla
df_results = pd.DataFrame(data)

# Mostrar la tabla
print(df_results)

In [ ]:
matriz_conf = confusion_matrix(y, y_hat)
print('MATRIZ DE CONFUSIÓN:')
print(matriz_conf,'\n')
print('BIEN CLASIFICADOS: ', matriz_conf[0][0]+matriz_conf[1][1])
print('\tPasajeros con sobrevivencia = 0 clasificados como 0: ', matriz_conf[0][0])
print('\tPasajeros con sobrevivencia = 1 clasificados como 1: ', matriz_conf[1][1])
print('MAL CLASIFICADOS: ', matriz_conf[0][1]+matriz_conf[1][0])
print('\tPasajeros con sobrevivencia = 0 clasificados como 1: ', matriz_conf[0][1],'(falso positivo)')
print('\tPasajeros con sobrevivencia = 1 clasificados como 0: ', matriz_conf[1][0],'(falso negativo)')

El resultado de nuestra predicción se ve mucho mejor. Sin embargo, <b>NO TODO LO QUE BRILLA ES ORO</b>, y en Ciencia de Datos este conocimiento ancestral es muy importante de tener presente. 

<b>VALIDACIÓN DE LOS RESULTADOS </b>

<b> Overfiting (Sobreajuste) </b>

Si observamos la estructura de árbol que se ha generado podemos ver que tiene muchos nodos, es bastante profunda y compleja. Esto significa que a cada nodo hoja está llegando unas pocas observaciones con conjuntos de características muy particulares. Son muy pocos los nodos terminales que contienen observaciones de las dos clases. Esto hace que la separación de ambas clases resulte casi completa. 

Pero, si nos entregaran los datos de un nuevo pasajero, cuyos datos no se conocían al momento de entrenar el árbol, ¿sería nuestro modelo capaz de clasificarlo correctamente?

Podemos especular que NO, dado que el árbol de decisión que tenemos genera tanta variedad de salidas que en la práctica <i>memorizó</i> los datos con que fue entrenado. El algoritmo construyó algo similar a un árbol binario de búsqueda, en el cual está codificado el conjunto de características de cada observación, asociado en el nodo terminal con su clase. Algunas observaciones que comparten características y tenían la misma clase quedaron juntas, pero en general es más fruto del azar que de una relación real entre las variables predictoras y la clase. A este fenómeno se le conoce como <b> Overfiting (Sobreajuste)</b>.

Suponga que para estudiar para una prueba usted aprende de memoria las respuestas de 100 preguntas, sin realmente entender la materia. Mientras le hagan alguna de esas 100 preguntas, responderá con mucha seguridad. Pero preguntas fuera de esas 100, aunque se refieran a los mismos conceptos, no las podrá responder.

Con Regresión Logística el overfiting no es mayor problema porque un clasificador lineal no tiene recursos para ajustar más allá de una línea recta. Pero en Ciencia de Datos jugamos con clasificadores más poderosos, y por lo mismo, más riesgosos de sobreajustar. 
<br> <img src=https://raw.githubusercontent.com/xibsked/menka/master/books/artificial-intelligence/54ddbef68d6ed8c916d6b785e36e868d1.png width="400"/>

<b>Entrenamiento y Testeo</b>

¿Cómo podemos evaluar si nuestro modelo está sobreajustado? Podemos entrenar una parte de la muestra, reservándonos otra parte de ella para hacer un test. Así, la capacidad predictora la evaluamos no usando las mismas observaciones con que entrenamos, sino observaciones totalmente nuevas para el modelo. 


In [ ]:
from sklearn.model_selection import train_test_split

# Cargamos y preprocesamos los datos usando el pipeline
X, y = preprocesar_titanic("Titanic.csv")

# Separamos la muestra al azar, 80% para entrenar y 20% para testear. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
clf = DecisionTreeClassifier(random_state=1234)
model = clf.fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(100,40))
tree.plot_tree(clf, feature_names=["clase", "género", "edad", 
                                   "conyugues-hermanos", "padres-hijos" , "tarifa"], 
               class_names=["No sobrevice", "Sí sobrevive"], filled=True)
plt.show()

In [ ]:
# En esta ocasión usamos la muestra de validación para evaluar el resultado.
y_hat = model.predict(X_test)
samp = random.sample(range(len(y_test)),20)
# Crear un diccionario con los resultados reales y predichos para la muestra
data = {
    'Resultado Real': [y[i] for i in samp],
    'Resultado Predicho': [y_hat[i] for i in samp]
}

# Convertir el diccionario a un DataFrame de pandas para mostrar en tabla
df_results = pd.DataFrame(data)

# Mostrar la tabla
print(df_results)

In [ ]:
matriz_conf = confusion_matrix(y_test, y_hat)
print('MATRIZ DE CONFUSIÓN:')
print(matriz_conf,'\n')
print('BIEN CLASIFICADOS: ', matriz_conf[0][0]+matriz_conf[1][1])
print('\tPasajeros con sobrevivencia = 0 clasificados como 0: ', matriz_conf[0][0])
print('\tPasajeros con sobrevivencia = 1 clasificados como 1: ', matriz_conf[1][1])
print('MAL CLASIFICADOS: ', matriz_conf[0][1]+matriz_conf[1][0])
print('\tPasajeros con sobrevivencia = 0 clasificados como 1: ', matriz_conf[0][1],'(falso positivo)')
print('\tPasajeros con sobrevivencia = 1 clasificados como 0: ', matriz_conf[1][0],'(falso negativo)')

Nuestro modelo predictivo tipo Árbol de Decisión no resultó tan bueno para clasificar observaciones nuevas, de lo que era clasificando las observaciones con las que se había entrenado. Esto es totalmente esperable, siempre será menos preciso con casos nuevos respecto a con los que entrenó. 

¿Cómo podemos evitar que nuestro árbol de decisión quede sobreajustado? Hasta ahora hemos dejado que crezca lo que le haga falta para separar los datos, formando grupos cada vez más pequeños. Podemos controlar la profundidad del árbol, para obtener un clasificador que, aunque no sea tan preciso con los datos de entrenamiento, haga una mejor generalización a los datos de prueba. 

In [ ]:
clf = DecisionTreeClassifier(random_state=1234, max_depth=2)
model = clf.fit(X_train, y_train)
plt.figure(figsize=(400,100))
tree.plot_tree(clf, feature_names=["clase", "género", "edad", 
                                   "conyugues-hermanos", "padres-hijos" , "tarifa"], 
               class_names=["No sobrevive", "Sí sobrevive"], filled=True)
plt.show()

In [ ]:
y_hat = model.predict(X_test)
samp = random.sample(range(len(y_test)),20)
# Crear un diccionario con los resultados reales y predichos para la muestra
data = {
    'Resultado Real': [y[i] for i in samp],
    'Resultado Predicho': [y_hat[i] for i in samp]
}

# Convertir el diccionario a un DataFrame de pandas para mostrar en tabla
df_results = pd.DataFrame(data)

# Mostrar la tabla
print(df_results)

In [ ]:
matriz_conf = confusion_matrix(y_test, y_hat)
print('MATRIZ DE CONFUSIÓN para profundidad 2:')
print(matriz_conf,'\n')
print('BIEN CLASIFICADOS: ', matriz_conf[0][0]+matriz_conf[1][1])
print('\tPasajeros con sobrevivencia = 0 clasificados como 0: ', matriz_conf[0][0])
print('\tPasajeros con sobrevivencia = 1 clasificados como 1: ', matriz_conf[1][1])
print('MAL CLASIFICADOS: ', matriz_conf[0][1]+matriz_conf[1][0])
print('\tPasajeros con sobrevivencia = 0 clasificados como 1: ', matriz_conf[0][1],'(falso positivo)')
print('\tPasajeros con sobrevivencia = 1 clasificados como 0: ', matriz_conf[1][0],'(falso negativo)')

Observamos que el mismo modelo pero con sólo dos niveles de profundidad ya generaliza mejor que si dejamos que sobre ajuste. 

Definamos la métrica <b><i>accuracy (exactitud)</i></b> como: 

$$ accuracy = \frac{correct\ predicted } {total\ cases} = \frac{TN + TP } {TN+TP+FN+FP}$$


In [ ]:
acc = (matriz_conf[0][0]+matriz_conf[1][1])/(matriz_conf[0][0]+matriz_conf[1][1]+
                                            matriz_conf[0][1]+matriz_conf[1][0])
print('La exactitud de validación del modelo con profundidad 2 es: {:.3f}'.format(acc))

<b>Optimización de Hiperparámetros</b>

Busquemos a continuación la profundidad ideal, en el sentido de obtener la mayor exactitud de generalización. Es decir, que al probar el modelo con datos de validación obtengamos el accuracy más alto. 

In [ ]:
# Incrementamos la profundidad para obtener validation accuracy, hasta alcanzar un máximo 
# o que la profundidad del árbol ya no aumente. 
max_deep_seen = 0
max_acc_seen = 0
best_deep = 0
for d in range(1,100):
    clf = DecisionTreeClassifier(random_state=1234, max_depth=d)
    model = clf.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    matriz_conf = confusion_matrix(y_test, y_hat)
    acc = (matriz_conf[0][0]+matriz_conf[1][1])/(matriz_conf[0][0]+matriz_conf[1][1]+
                                            matriz_conf[0][1]+matriz_conf[1][0])
    deep = model.get_depth()
    if acc > max_acc_seen:
        max_acc_seen = acc
        best_deep = deep
    if deep > max_deep_seen:
        max_deep_seen = deep
    else:
        break

print('La profundidad que logró mayor exactitud es: ',best_deep)   

In [ ]:
# Entrenamos el modelo y lo validamos una vez más. 
clf = DecisionTreeClassifier(random_state=1234, max_depth=best_deep)
model = clf.fit(X_train, y_train)
plt.figure(figsize=(400,100))
tree.plot_tree(clf, feature_names=["clase", "género", "edad", 
                                   "conyugues-hermanos", "padres-hijos" , "tarifa"], 
               class_names=["No sobrevice", "Sí sobrevive"], filled=True)
plt.show()
y_hat = model.predict(X_test)
matriz_conf = confusion_matrix(y_test, y_hat)
acc = (matriz_conf[0][0]+matriz_conf[1][1])/(matriz_conf[0][0]+matriz_conf[1][1]+
                                            matriz_conf[0][1]+matriz_conf[1][0])
print('MATRIZ DE CONFUSIÓN para profundidad {}:'.format(best_deep))
print(matriz_conf,'\n')
print('BIEN CLASIFICADOS: ', matriz_conf[0][0]+matriz_conf[1][1])
print('\tPasajeros con sobrevivencia = 0 clasificados como 0: ', matriz_conf[0][0])
print('\tPasajeros con sobrevivencia = 1 clasificados como 1: ', matriz_conf[1][1])
print('MAL CLASIFICADOS: ', matriz_conf[0][1]+matriz_conf[1][0])
print('\tPasajeros con sobrevivencia = 0 clasificados como 1: ', matriz_conf[0][1],'(falso positivo)')
print('\tPasajeros con sobrevivencia = 1 clasificados como 0: ', matriz_conf[1][0],'(falso negativo)')
print('La exactitud de validación del modelo con profundidad {} es: {:.3f}'.format(best_deep,acc))


Por lo tanto, si quisiéramos reportar el mejor modelo encontrado para que sea usado en predicción de nuevos casos que no estaban en la muestra original, lo que haríamos a continuación es entrenar el árbol de decisión con la totalidad de los datos (sin reservar para validación) puesto que ese es el máximo de información que tenemos disponible. Podemos esperar que el modelo que resulte será, a lo menos, tan bueno como el que hemos validado previamente. 

<b>ENTRENAMIENTO, VALIDACIÓN Y TESTEO </b>

Resumamos lo que hemos visto en este apunte: estudiamos los dos problemas fundamentales para los que se usa el aprendizaje supervisado: <b>REGRESIÓN</b> y <b>CLASIFICACIÓN</b>

<br> <img src=https://www.simplilearn.com/ice9/free_resources_article_thumb/Regression_vs_Classification.jpg width="400"/>


Vimos que para resolver un problema de Clasificación se podría usar, en primera instancia, una <b>Regresión Logística</b>, que si bien es un modelo de regresión, gracias a la incorporación de la función logística se transforma en un modelo de clasificación lineal. Dijimos que era un clasificador lineal porque la frontera sobre la que decide a qué clase corresponde cada observación es una línea recta, y eso imponía una restricción al modelo. Un clasificador lineal no puede separar clases que no son linealmente separables, su capacidad de representación se limita a problemas lineales. En el problema de los sobrevivientes del Titanic, por ejemplo, Regresión Logística no lograba separar ambas clases por completo. 

Entonces decidimos buscar un modelo con la capacidad de representar fronteras no lineales entre las clases. Árboles de Decisión, al formar una topología de árbol, puede fraccionar los datos en las áreas que haga falta para separar las clases, casi por completo. Observaciones muy similares en todos sus aspectos ($X$), que sólo difieran en su clase ($y$) no logran ser separados por un árbol de decisión suficientemente profundos. Pero vimos que esto también es un problema, debido al <b>Overfitting</b>, es decir la capacidad del modelo a sobre-ajustarse a la forma de los datos.

<br> <img src=https://media.geeksforgeeks.org/wp-content/cdn-uploads/20190523171258/overfitting_2.png width="400"/>

Vimos que las consecuencias negativas del overfitting se aprecian al tratar de clasificar observaciones distintas de aquellas con las que se entrenó. Nuestro árbol sobre-ajustado siempre logra clasificar con alta exactitud a esas observaciones con las que fue entrenado, pero fallaba en clasificar casos distintos. Ello lo pudimos observar cuando dividimos la muestra en un 80% para <i>entrenamiento</i> y 20% para <i>testeo</i>. 

<br> <img src=https://upload.wikimedia.org/wikipedia/commons/b/bb/ML_dataset_training_validation_test_sets.png width="400"/>

En la imagen anterior aparece, además, una tercera partición de datos para usar en <b><i>validación</i></b>. La idea es que usaremos la muestra de entrenamiento para alimentar el modelo, la muestra de validación para seleccionar los hiperparámetros ideales del modelo, por ejemplo la mejor profundidad del árbol, y luego reportaremos los resultados del modelo usando la muestra de testeo. 

Nosotros habíamos reportado la exactitud midiendo sobre el mismo set de datos con que elegimos la profundidad del modelo, y con eso estamos generando condiciones de sobre-ajuste del modelo. Entonces, lo que haremos a continuación es separar los datos en estos tres conjuntos, 60% lo usaremos para entrenar, 20% para encontrar la mejor configuración y 20% para evaluar los resultados. 

Repitamos el experimento una vez más, pero usando el set de validación en el paso de encontrar la profundidad ideal. 

In [ ]:
from sklearn.model_selection import train_test_split

# Cargamos y preprocesamos los datos usando el pipeline
X, y = preprocesar_titanic("Titanic.csv")

# Separamos la muestra al azar, 60% para entrenar, 20% para validación y 20% para testeo final. 
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.4, random_state=123)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=123)

# Incrementamos la profundidad para obtener testing accuracy, hasta alcanzar un máximo 
# o que la profundidad del árbol ya no aumente. 
max_deep_seen = 0
max_acc_seen = 0
best_deep = 0
for d in range(1,100):
    clf = DecisionTreeClassifier(random_state=1234, max_depth=d)
    # Entrenamos con el set de datos de entrenamiento
    model = clf.fit(X_train, y_train)
    # Y evaluamos con el set de validación para elegir la profundida adecuada
    y_hat = model.predict(X_val)
    matriz_conf = confusion_matrix(y_val, y_hat)
    acc = (matriz_conf[0][0]+matriz_conf[1][1])/(matriz_conf[0][0]+matriz_conf[1][1]+
                                            matriz_conf[0][1]+matriz_conf[1][0])
    deep = model.get_depth()
    if acc > max_acc_seen:
        max_acc_seen = acc
        best_deep = deep
    if deep > max_deep_seen:
        max_deep_seen = deep
    else:
        break
print('La profundidad que logró mayor exactitud es: ',best_deep)   
# Entrenamos el modelo y lo testeamos una vez más. 
clf = DecisionTreeClassifier(random_state=1234, max_depth=best_deep)
model = clf.fit(X_train, y_train)
plt.figure(figsize=(400,100))
tree.plot_tree(clf, feature_names=["clase", "género", "edad", 
                                   "conyugues-hermanos", "padres-hijos" , "tarifa"], 
               class_names=["No sobrevice", "Sí sobrevive"], filled=True)
plt.show()

# Sólo acá usamos el set de datos de testing
y_hat = model.predict(X_test)
matriz_conf = confusion_matrix(y_test, y_hat)
acc = (matriz_conf[0][0]+matriz_conf[1][1])/(matriz_conf[0][0]+matriz_conf[1][1]+
                                            matriz_conf[0][1]+matriz_conf[1][0])
print('MATRIZ DE CONFUSIÓN para profundidad {}:'.format(best_deep))
print(matriz_conf,'\n')
print('BIEN CLASIFICADOS: ', matriz_conf[0][0]+matriz_conf[1][1])
print('\tPasajeros con sobrevivencia = 0 clasificados como 0: ', matriz_conf[0][0])
print('\tPasajeros con sobrevivencia = 1 clasificados como 1: ', matriz_conf[1][1])
print('MAL CLASIFICADOS: ', matriz_conf[0][1]+matriz_conf[1][0])
print('\tPasajeros con sobrevivencia = 0 clasificados como 1: ', matriz_conf[0][1],'(falso positivo)')
print('\tPasajeros con sobrevivencia = 1 clasificados como 0: ', matriz_conf[1][0],'(falso negativo)')
print('La exactitud de testeo del modelo con profundidad {} es: {:.3f}'.format(best_deep,acc))

Vale la pena mencionar que podemos obtener la exactitud del modelo preguntándosela directamente, en lugar de computarla a partir de la matriz de confusión. El método <i>score(X,y)</i> en los modelos de clasificación, se asocia a la métrica accuracy. Por simpleza lo haremos así en adelante. 

In [ ]:
print('Accuracy: ',model.score(X_test,y_test))

Si repetimos el experimento reproduciremos una vez más el mismo resultado, pese a que parte de nuestro proceso ha sido aleatorio, y eso se debe al uso de seed (en este caso hiperparámetro random_state) que fija la generación de números aleatorios. En particular tenemos dos elementos de aleatoriedad: (1) el algoritmo que genera los árboles de decisión sigue un proceso aleatorio en la búsqueda de los atributos que mejor separan las clases, no siempre va a encontrar lo mismo; y (2) el set de datos se separó aleatoriamente entre testeo y entrenamiento. 

Respecto a la primera aleatoriedad, la que es propia del modelo, nos interesa mantenerla "fija" dado que queremos comparar distintas configuraciones, y saber si un cambio mejora o empeora el resultado, para lo cual necesitamos que el cambio quede aislado de toda otra modificación. Si hubiera una componente de aleatoriedad en los resultados al comparar dos configuraciones, se haría más complejo el análisis y decisión respecto al mejor modelo. 

Por otra parte, enfoquémonos en la aleatoriedad del set de datos al dividirse en entrenamiento y testeo. ¿Qué tan distinto podría haber sido nuestra conclusión con una distribución de datos diferente? Averigüémoslo repitiendo el experimento sin usar el hiperparámetro random_state al momento de hacer el split de datos. 

In [ ]:
# Cargamos y preprocesamos los datos usando el pipeline
X, y = preprocesar_titanic("Titanic.csv")

# Separamos la muestra al azar, pero esta vez sin seed (sin random_state). 
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.4)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5)

# Incrementamos la profundidad para obtener testing accuracy, hasta alcanzar un máximo 
# o que la profundidad del árbol ya no aumente. 
max_deep_seen = 0
max_acc_seen = 0
best_deep = 0
for d in range(1,100):
    clf = DecisionTreeClassifier(random_state=1234, max_depth=d)
    # Entrenamos con el set de datos de entrenamiento
    model = clf.fit(X_train, y_train)
    # Y evaluamos con el set de validación para elegir la profundida adecuada
    y_hat = model.predict(X_val)
    matriz_conf = confusion_matrix(y_val, y_hat)
    acc = (matriz_conf[0][0]+matriz_conf[1][1])/(matriz_conf[0][0]+matriz_conf[1][1]+
                                            matriz_conf[0][1]+matriz_conf[1][0])
    deep = model.get_depth()
    if acc > max_acc_seen:
        max_acc_seen = acc
        best_deep = deep
    if deep > max_deep_seen:
        max_deep_seen = deep
    else:
        break
print('La profundidad que logró mayor exactitud es: ',best_deep)   
# Entrenamos el modelo y lo testeamos una vez más. 
clf = DecisionTreeClassifier(random_state=1234, max_depth=best_deep)
model = clf.fit(X_train, y_train)
plt.figure(figsize=(400,100))
tree.plot_tree(clf, feature_names=["clase", "género", "edad", 
                                   "conyugues-hermanos", "padres-hijos" , "tarifa"], 
               class_names=["No sobrevice", "Sí sobrevive"], filled=True)
plt.show()

# Sólo acá usamos el set de datos de testing
y_hat = model.predict(X_test)
matriz_conf = confusion_matrix(y_test, y_hat)
acc = (matriz_conf[0][0]+matriz_conf[1][1])/(matriz_conf[0][0]+matriz_conf[1][1]+
                                            matriz_conf[0][1]+matriz_conf[1][0])
print('MATRIZ DE CONFUSIÓN para profundidad {}:'.format(best_deep))
print(matriz_conf,'\n')
print('BIEN CLASIFICADOS: ', matriz_conf[0][0]+matriz_conf[1][1])
print('\tPasajeros con sobrevivencia = 0 clasificados como 0: ', matriz_conf[0][0])
print('\tPasajeros con sobrevivencia = 1 clasificados como 1: ', matriz_conf[1][1])
print('MAL CLASIFICADOS: ', matriz_conf[0][1]+matriz_conf[1][0])
print('\tPasajeros con sobrevivencia = 0 clasificados como 1: ', matriz_conf[0][1],'(falso positivo)')
print('\tPasajeros con sobrevivencia = 1 clasificados como 0: ', matriz_conf[1][0],'(falso negativo)')
print('La exactitud de testeo del modelo con profundidad {} es: {:.3f}'.format(best_deep,acc))

De hecho, no sé qué dará cuando lo ejecute en clase, pero es muy probable que lleguemos a una profundidad de árbol distinta, así como a un <i>testing accuracy</i> diferente, respecto a lo que habíamos reportado antes. Además, si repetimos el experimento varias veces, observaremos resultados muy diferentes en cada ocasión. 

¿Qué resultado es más representativo de la realidad?, si vamos a construir un modelo predictivo para usar en un ambiente real, ¿cómo definimos de qué profundidad debiera ser el árbol?

<b>VALIDACIÓN CRUZADA</b>

Si repitiéramos el experimento anterior varias veces, fuéramos registrando la profundidad de árbol y exactitud para distintas distribuciones de entrenamiento y validación, podríamos promediar los resultados de exactitud para cada potencial profundidad, y usar eso para identificar la profundidad ideal así como una estimación de qué tan exacto podemos esperar que sea el modelo resultante. Esta idea es la base de una de las técnicas de validación más recurrentemente utilizada en Ciencia de Datos: <b>Cross Validation (Validación Cruzada)</b>.

El set de datos se divide en un número $n$ de particiones, que llamaremos carpetas (folds), y a continuación se realiza el experimento $n$ veces tomando como set de validación una de esas carpetas, sucesivamente. En cada iteración se dispone de $n-1$ carpetas para entrenar y una para validar los resultados. Al finalizar se promedia los resultados de las $n$ iteraciones, y podemos asumir que eso es representativo de lo que pasaría si nos llegara nuevos datos para realizar predicción. 

<br> <img src=https://miro.medium.com/v2/resize:fit:1200/1*AAwIlHM8TpAVe4l2FihNUQ.png width="400"/>

Al finalizar el proceso sabemos que cada observación participó en el entrenamiento de $n-1$ modelos y sirvió para validar los resultados de 1 modelo. Con ello deja de ser crítico en el resultado lo que dicte el azar al momento de generar la distribución de los datos. 

Cuando quisimos saber qué tan exacto era el árbol con profundidad 2, usamos un 80% de la muestra para entrenar y 20% para testearlo. Acabamos de ver que, además de eso, tenemos que apartar otro 20% para validación, reduciendo aún más la cantidad disponible para entrenamiento. Vimos también en esta clase que al modificar la distribución de la muestra al momento de hacer el split cambia el modelo al que llegamos y su exactitud. 

Usando Validación Cruzada podemos evitarnos disponer de una muestra de validación, y además obtendremos resultados más estables. Probemos. 

In [ ]:
from sklearn.model_selection import cross_validate

# Cargamos y preprocesamos los datos usando el pipeline
X, y = preprocesar_titanic("Titanic.csv")

# Instanciamos el clasificador con profundidad 2
clf = DecisionTreeClassifier(random_state=1234, max_depth=2)

# Usamos 5 carpetas de validación cruzada para evaluar nuestro clasificador. 
cv_results = cross_validate(clf, X, y, cv=5)

# ¿Qué tenemos guardado como resultados?
print(cv_results)

Los resultados son un diccionario con dos métricas asociadas al tiempo de entrenamiento y de evaluación, mientras que con la clave 'test_score' obtenemos la métrica asociada al puntaje, que en modelos de clasificación es la exactitud o accuracy. Es sobre esta última que necesitamos conocer un valor promedio. 

In [ ]:
print('La exactitud promedio (VC) del árbol con profundidad 2 es: ', cv_results['test_score'].mean())

Entonces, ¿cuál es la profundidad ideal del árbol para el problema del Titanic? Repitamos la búsqueda exhaustiva pero esta vez usaremos la exactitud promedio obtenida en validación cruzada, en lugar de exactitud de testeo. 

In [ ]:
# Cargamos y preprocesamos los datos usando el pipeline
X, y = preprocesar_titanic("Titanic.csv")

# Incrementamos la profundidad para obtener testing accuracy, hasta alcanzar un máximo 
# o que la profundidad del árbol ya no aumente. 
max_deep_seen = 0
max_acc_seen = 0
best_deep = 0
for d in range(1,100):
    clf = DecisionTreeClassifier(random_state=1234, max_depth=d)
    cv_results = cross_validate(clf, X, y, cv=5)
    acc = cv_results['test_score'].mean()
    if acc > max_acc_seen:
        max_acc_seen = acc
        best_deep = d
print('La profundidad que logró mayor exactitud promedio es: ',best_deep)
print('La exactitud promedio (VC) obtenida con profundidad {} es {:.3f}'.format(best_deep,max_acc_seen))

Podemos preguntarnos qué tan estable es este resultado ante cambios en la distribución de las carpetas, dado que ya vimos que con distintas distribuciones entre entrenamiento y testeo el resultado no era nada de estable. Acá no hay un elemento de aleatoriedad que podamos directamente manipular al hacer el split entre carpetas, dado que cross validation lo que hace es dividir el set de datos en segmentos iguales, pero manteniendo el orden en que viene. Si queremos una distribución diferente tendríamos que barajar los datos antes de aplicar la validación cruzada... Hagamos eso. 

In [ ]:
import numpy as np

# Cargamos y preprocesamos los datos usando el pipeline
X, y = preprocesar_titanic("Titanic.csv")

# Obtenemos una permutación aleatoria de los índices
indices = np.random.permutation(len(X))

# Reordenamos X e y con la misma permutación
X_barajado = X[indices]
y_barajado = y[indices]

Note que es importante que el barajado de los vectores X e y se haga en forma coordinada, no queremos perder la relación posicional que define los atributos X e y para cada observación. 

In [ ]:
# Continuamos con la búsqueda exhaustiva de la mejor profundidad. 
max_deep_seen = 0
max_acc_seen = 0
best_deep = 0
for d in range(1,100):
    clf = DecisionTreeClassifier(random_state=1234, max_depth=d)
    cv_results = cross_validate(clf, X_barajado, y_barajado, cv=5)
    acc = cv_results['test_score'].mean()
    if acc > max_acc_seen:
        max_acc_seen = acc
        best_deep = d
print('La profundidad que logró mayor exactitud promedio es: ',best_deep)
print('La exactitud promedio (VC) obtenida con profundidad {} es {:.3f}'.format(best_deep,max_acc_seen))

Nuevamente no sé qué va a dar cuando lo ejecutemos en clase, pero podemos esperar que no sea exactamente el mismo resultado que antes. Si repetimos el experimento varias veces, debiéramos ver resultados variables pero, con mucho menos dispersión que la que teníamos antes, cuando sólo había una muestra de testeo. 

Hemos ganado en precisar el rango en el que se movería la profundidad óptima y la exactitud esperable del modelo. Sin embargo, todavía quedan otros hiperparámetros del algoritmo que construye el árbol de decisión, que por ahora no hemos tocado. 

<br> <img src=https://pbs.twimg.com/media/ETBrXfgXkAAIFGp.jpg width="400"/>

<b> CONCLUSIÓN </b>

Este apunte ha recorrido tópicos de Ciencia de Datos relacionados con Aprendizaje Supervisado. Se ha presentado el problema de Regresión y Clasificación desde sus fundamentos y técnicas estadísticas. Profundizando en clasificación, se ha señalado la limitación intrínseca de usar un clasificador lineal, como es Regresión Logística. Se ha presentado los modelos de Árbol de Decisión como una alternativa de clasificador no lineal. Se ha identificado metodología y métricas adecuadas para la evaluación de los modelos de clasificación, incluido el separar muestras de entrenamiento, validación y testeo. Se ha observado el efecto negativo de sobreajustar, dada la mayor complejidad del modelo.  Se ha aplicado una medida de mitigación al riesgo de sobreajuste, controlando la complejidad del modelo. Se ha recorrido el dominio de un hiperparámetro del algoritmo buscando la configuración más exitosa en clasificar una muestra de validación, reportando los resultados con una muestra de testeo.   

Este apunte ha sido un recorrido amplio sobre diversos temas. La intención es facilitar la comprensión general del quehacer en Ciencia de Datos, pero por lo mismo ha dejado varios temas sin suficiente exploración. En próximas clases seguiremos profundizando sobre las métricas y algoritmos involucrados en la construcción de los modelos, en qué consiste el entrenamiento y sobre qué familias de modelos podríamos usar. 

Este apunte contiene bastante material práctico en Python para la Ciencia de Datos, por lo que debiera servir tanto para el desarrollo de talleres como también de material de estudio teórico. Se sugiere no sólo ejecutar el código, sino sobre todo entenderlo, asimilarlo y apropiárselo. Procure relacionar el código con las ecuaciones, explicaciones y visualizaciones que le acompañan. 